In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import datetime
from sklearn import preprocessing
sns.set()

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# train.info()

In [3]:
# train.isnull().sum()

In [4]:
from datetime import datetime

train['DOB'] = pd.to_datetime(train.DOB)
train['month'] = train['DOB'].dt.month
# train['dayofweek'] = train['DOB'].dt.dayofweek
train['year'] = train['DOB'].dt.year
# train['day'] = train['DOB'].dt.day
# train['dayofyear'] = train['DOB'].dt.dayofyear
# train['weekofyear'] = train['DOB'].dt.weekofyear


test['DOB'] = pd.to_datetime(test.DOB)
test['month'] = test['DOB'].dt.month
# test['dayofweek'] = test['DOB'].dt.dayofweek
test['year'] = test['DOB'].dt.year
# test['day'] = test['DOB'].dt.day
# test['dayofyear'] = test['DOB'].dt.dayofyear
# test['weekofyear'] = test['DOB'].dt.weekofyear

train.drop(['DOB'],axis=1,inplace=True)
test.drop(['DOB'],axis=1,inplace=True)



In [5]:
for index,row in train.iterrows(): 
    if(row['lump_sum_pay']>20000):
        train.drop(index,inplace=True)

In [6]:
train['train']=1
test['train']=0

In [7]:
temp=pd.concat([train,test])

/home/freakcap/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [8]:
labelencoder = LabelEncoder()
# temp['location'] = labelencoder.fit_transform(temp['location'])
temp['employer'] = labelencoder.fit_transform(temp['employer'])
# temp['pension'] = labelencoder.fit_transform(temp['pension'])
# temp['employer_url'] = labelencoder.fit_transform(temp['employer_url'])

In [9]:
temp['lump_sum_pay'].fillna(temp.groupby('employer')['lump_sum_pay'].transform('mean'), inplace=True)
# temp['location'].fillna(temp.groupby('employer')['location'].transform('value_counts'), inplace=True)
temp['retirement_cost_covered'].fillna(temp.groupby('employer')['retirement_cost_covered'].transform('mean'), inplace=True)
# temp['pension'].fillna(temp.groupby('employer')['lump_sum_pay'].transform('mean'), inplace=True)


In [10]:
train = temp[temp['train']==1]
test = temp[temp['train']==0]
train.drop(['train'],axis=1,inplace=True)
test.drop(['train'],axis=1,inplace=True)
test.drop(['total_wages'],axis=1,inplace=True)


/home/freakcap/anaconda3/envs/ml/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
train.drop(['location'],axis=1,inplace=True)
train.drop(['pension'],axis=1,inplace=True)
train.drop(['employer_url'],axis=1,inplace=True)
train.drop(['Id'],axis=1,inplace=True)


test.drop(['location'],axis=1,inplace=True)
test.drop(['pension'],axis=1,inplace=True)
test.drop(['employer_url'],axis=1,inplace=True)

Id = test['Id']

test.drop(['Id'],axis=1,inplace=True)


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22955 entries, 0 to 22999
Data columns (total 11 columns):
employer                   22955 non-null int64
health_benefits            22955 non-null float64
lump_sum_pay               22955 non-null float64
max_pos_salary             22955 non-null float64
min_pos_salary             22955 non-null float64
month                      22955 non-null int64
other_pay                  22955 non-null float64
overtime_pay               22955 non-null float64
retirement_cost_covered    22955 non-null float64
total_wages                22955 non-null float64
year                       22955 non-null int64
dtypes: float64(8), int64(3)
memory usage: 2.1 MB


In [13]:
for index,row in train.iterrows(): 
    if(row['min_pos_salary']>100000 or row['total_wages']>250000):
        train.drop(index,inplace=True)
    elif(row['max_pos_salary']>350000):
        train.drop(index,inplace=True)
    elif(row['overtime_pay']>20000):
        train.drop(index,inplace=True)
    elif(row['other_pay']>40000):
        train.drop(index,inplace=True)
    elif(row['retirement_cost_covered']>500):
        train.drop(index,inplace=True)
    elif(row['health_benefits']<-5000):
        train.drop(index,inplace=True)

In [14]:
y = train["total_wages"]

X = train.drop(["total_wages"],axis=1)

In [15]:
from sklearn.model_selection import train_test_split 

x_train , x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

X_train = x_train
X_test = x_test

In [16]:
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 100)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs=-1)# Fit the random search model
rf_random.fit(X, y)


{'n_estimators': [200, 650, 1100, 1550, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}
Fitting 2 folds for each of 50 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

In [ ]:
rf_random.best_params_